# Performance Test of `sedkit`
Th purpose of this notebook is to test the performance of `sedkit` across the main sequence by comparing the results of this software against well characterized stars in the literature.

In [204]:
from sedkit.uncertainties import Unum, UArray
from astropy import units as q
from bokeh.io import output_notebook
import numpy as np
output_notebook()

# Data structures
u1 = Unum(10*q.um, 1*q.um, n_samples=200)
u2 = Unum(10, 1, n_samples=20)
a1 = UArray(np.ones(3)*q.um, np.abs(np.random.normal(size=3))*q.um, n_samples=1000)
a2 = UArray(np.ones(3), np.abs(np.random.normal(size=3)), n_samples=1000)
i1 = 5*q.um
i2 = 5
s1 = np.array([1, 2, 3])*q.um
s2 = np.array([1, 2, 3])

# Addition
u1 + u1
u1 + i1
u2 + u2
u2 + i2
a1 + u1
a1 + i1
a1 + s1
a2 + u2
a2 + i2
a2 + s2

# Subtraction
u1 - u1
u1 - i1
u2 - u2
u2 - i2
a1 - u1
a1 - i1
a1 - s1
a2 - u2
a2 - i2
a2 - s2

# Multiplication
u1 * u1
u1 * u2
u1 * i1
u1 * i2
u1 * s1
u1 * s2
u1 * a1
u1 * a2

u2 * u1
u2 * u2
u2 * i1
u2 * i2
u2 * s1
u2 * s2
u2 * a1
u2 * a2

a1 * u1
a1 * u2
a1 * i1
a1 * i2
a1 * s1
a1 * s2
a1 * a1
a1 * a2

a2 * u1
a2 * u2
a2 * i1
a2 * i2
a2 * s1
a2 * s2
a2 * a1
a2 * a2

# TrueDiv
u1 / u1
u1 / u2
u1 / i1
u1 / i2
u1 / s1
u1 / s2
u1 / a1
u1 / a2

u2 / u1
u2 / u2
u2 / i1
u2 / i2
u2 / s1
u2 / s2
u2 / a1
u2 / a2

a1 / u1
a1 / u2
a1 / i1
a1 / i2
a1 / s1
a1 / s2
a1 / a1
a1 / a2

a2 / u1
a2 / u2
a2 / i1
a2 / i2
a2 / s1
a2 / s2
a2 / a1
a2 / a2

# FloorDiv
u1 // u1
u1 // u2
u1 // i1
u1 // i2
u1 // s1
u1 // s2
u1 // a1
u1 // a2

u2 // u1
u2 // u2
u2 // i1
u2 // i2
u2 // s1
u2 // s2
u2 // a1
u2 // a2

a1 // u1
a1 // u2
a1 // i1
a1 // i2
a1 // s1
a1 // s2
a1 // a1
a1 // a2

a2 // u1
a2 // u2
a2 // i1
a2 // i2
a2 // s1
a2 // s2
a2 // a1
a2 // a2

# Exponential
u1**2
u2**2
a1**2
a2**2

# Log10
u2.log10()
a2.log10()

# Polynomial
u1.polyval([1, 2, 3])
u2.polyval([1, 2, 3])
a1.polyval([1, 2, 3])
a2.polyval([1, 2, 3])

Loading BokehJS ...

ValueError: operands could not be broadcast together with shapes (1000,) (3,) 

In [159]:
from sedkit import SED, Catalog, BTSettl
from astropy import units as q
from astroquery.xmatch import XMatch
from multiprocessing.pool import ThreadPool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Stars with spectra
For this test, I crossmatch Gaia DR2 with the SDSS APOGEE 2 survey to get a good distribution of stars with parallaxes, NIR spectra, and effective temperatures. 

In [ ]:
# Crossmatch surveys... this takes about a minute to find 475k sources
all_sources = XMatch.query(cat1='vizier:III/284/allstars', cat2='vizier:II/246/out', max_distance=5 * q.arcsec)

In [ ]:
# Take a random sample of 100 sources wth effective temperatures
n_sources = 10
cols = ['Gaia', 'Teff', 'e_Teff', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'e_Kmag']
sources = all_sources[all_sources['Teff'] > 100]

# Sort by Teff then use sample spread across Teff space
sources.sort('Teff')
skip = int(len(sources) / n_sources)
sources = sources[::skip][cols]
sources.pprint(max_width=-1, max_lines=-1)

In [ ]:
# Function for multiprocessing
bt = BTSettl()
row = sources[1]
s = SED('Gaia DR2 {}'.format(row[0]), verbose=True)
s.use_best_fit = True

# Add the 2MASS photometry
s.add_photometry('2MASS.J', row[3], row[4])
s.add_photometry('2MASS.H', row[5], row[6])
s.add_photometry('2MASS.Ks', row[7], row[8])

# Get SDSS photometry
s.find_SDSS()

# Get WISE photometry
s.find_WISE()

# Get APOGEE spectrum
s.find_SDSS_spectra(surveys=['optical', 'apogee'], search_radius=None)

# Get Gaia parallax
s.find_Gaia(include=['parallax'])
s.Teff_Gaia = row[1]*q.K, row[2]*q.K

# Fit model grid
s.fit_modelgrid(bt, fit_to='phot')

s.plot()
s.results

In [ ]:
from sedkit.uncertainties import UArray
m = UArray(np.abs(np.random.normal(size=200))*q.um, np.abs(np.random.normal(size=200))*q.um)
# print(m.quantity)
x = m * Unum(10*q.um, 0.1*q.um)

In [ ]:
# Function for multiprocessing
bt = BTSettl()
def sed_pipeline(row, model=bt):
    # Make the SED
    s = SED('Gaia DR2 {}'.format(row[0]), verbose=False)

    # Add the 2MASS photometry
    s.add_photometry('2MASS.J', row[3], row[4])
    s.add_photometry('2MASS.H', row[5], row[6])
    s.add_photometry('2MASS.Ks', row[7], row[8])

    # Get WISE photometry
    s.find_WISE()
    
    # Get APOGEE spectrum
    s.find_SDSS_spectra(surveys=['optical', 'apogee'], search_radius=None)
    
    # Fit model grid
    s.fit_modelgrid(model, fit_to='phot')

    # Get Gaia parallax
    s.find_Gaia(include=['parallax'])
    s.Teff_Gaia = row[1]*q.K, row[2]*q.K

    s.make_sed()

    return s

In [281]:
s = [1, 2, 3]
x = s.pop()
print(x, s)

3 [1, 2]


In [ ]:
# Multiprocess sources table    
pool = ThreadPool(8)
all_seds = pool.map(sed_pipeline, sources.iterrows())
pool.close()
pool.join()

In [ ]:
# Make a catalog
cat = Catalog('Gaia + APOGEE')

# Add the SEDs to the catalog
for s in all_seds:
    cat.add_SED(s)

# Add Gaia Teff column
cat.add_column('Teff_Gaia', list(sources['Teff'])*q.K, unc=list(sources['e_Teff'])*q.K)

# Make a scatter plot
from bokeh.plotting import figure, show
fig = figure()
fig.circle(cat.results['Teff'], cat.results['Teff_Gaia'])
show(fig)

In [ ]:
cat.results['name', 'Teff', 'Teff_Gaia']

In [ ]:
from astroquery.simbad import Simbad
Simbad.cache_location

In [ ]:
s.photometry